In [16]:
# import needed modules
import time, random

# define our functions
def inc(x):
    time.sleep(random.random())
    return x + 1

def dec(x):
    time.sleep(random.random())
    return x - 1

def add(x, y):
    time.sleep(random.random())
    return x + y

In [17]:
%%time

# a sequential example with no parallelization
results = []
for x in range(20):
    result = inc(x)
    result = dec(result)
    results.append(result)

print(results)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
CPU times: user 3.4 ms, sys: 92 µs, total: 3.49 ms
Wall time: 17.4 s


In [18]:
# import dask for parallel work
from dask.distributed import Client, LocalCluster

In [19]:
# Set up our cluster with default workers and threads
cluster = LocalCluster(processes=False)
cluster

LocalCluster(4eda7393, 'inproc://10.244.1.209/1016/68', workers=1, threads=4, memory=1.85 GiB)

In [20]:
%%time
# Set up a client for work
client = cluster.get_client()

results = []
for x in range(20):
    result = client.submit(inc, x)
    result = client.submit(dec, result)
    results.append(result)

results = client.gather(results)
print(results)
client.close()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
CPU times: user 450 ms, sys: 63 ms, total: 513 ms
Wall time: 6.01 s


In [21]:
# When we are done we can close our dask cluster
cluster.close()

In [22]:
# Set up a new cluster with default 4 workers and 1 thread per worker
cluster = LocalCluster(n_workers=4, processes=False, threads_per_worker=1)
cluster

LocalCluster(f357e46f, 'inproc://10.244.1.209/1016/77', workers=4, threads=4, memory=1.85 GiB)

In [23]:
%%time
# Set up a client for work
client = cluster.get_client()

results = []
for x in range(20):
    result = client.submit(inc, x)
    result = client.submit(dec, result)
    results.append(result)

results = client.gather(results)
print(results)
client.close()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
CPU times: user 500 ms, sys: 55 ms, total: 555 ms
Wall time: 4.47 s


In [24]:
# When we are done we can close our dask cluster
cluster.close()